Notebook for project 3.

### Import data: client

In [ ]:
import pandas as pd
import numpy as np

client = pd.read_csv("../data/CLIENT_191102.tsv", encoding='utf-8',delimiter='\t')
client.head()


Clean data by checking values in each column.

In [ ]:
client.groupby("EE Provider ID").size()


For this project, we only keep data from "Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)" because assumedly others are closed in 2015.

In [ ]:
client=client[client["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']

In [ ]:
# get the number of unique values of "Client Unique ID"
client["Client Unique ID"].nunique()

In [ ]:
# get the number of unique values of "Client Unique ID"
client["Client ID"].nunique()

We have more unique "Client ID" than unique "Client Unique ID" here, so we will use "Client ID" instead of "Client Unique ID".

In [ ]:
#drop the "Client Unique ID" and "EE Provider ID"

client = client.drop(['Client Unique ID', 'EE Provider ID'],1)
client.head()

In [ ]:
client.groupby("Client Age at Entry").size()

In [ ]:
client.groupby("Client Age at Exit").size()

In [ ]:
client.groupby("Client Gender").size()

In [ ]:
client.groupby("Client Primary Race").size()

In [ ]:
#change "Client doesn't know" "Client refused" and "Data not collected" to NaN
client['Client Primary Race']=client['Client Primary Race'].replace("Client doesn't know (HUD)", np.NaN).replace("Client refused (HUD)", np.NaN).replace("Data not collected (HUD)", np.NaN)
client.groupby("Client Primary Race").size()

In [ ]:
client.groupby("Client Ethnicity").size()

In [ ]:
client['Client Ethnicity']=client['Client Ethnicity'].replace("Client doesn't know (HUD)", np.NaN).replace("Client refused (HUD)", np.NaN).replace("Data not collected (HUD)", np.NaN)
client.groupby("Client Ethnicity").size()

In [ ]:
client.groupby("Client Veteran Status").size()

In [ ]:
client['Client Veteran Status']=client['Client Veteran Status'].replace("Data not collected (HUD)", np.NaN)
client.groupby("Client Veteran Status").size()

### Import data: entry_exit

In [ ]:
entry_exit = pd.read_csv("../data/ENTRY_EXIT_191102.tsv", encoding='utf-8',delimiter='\t')
entry_exit.head()

Clean data: basicly the same thing as client data

In [ ]:
entry_exit.groupby("EE Provider ID").size()
entry_exit=entry_exit[entry_exit["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']

In [ ]:
entry_exit = entry_exit.drop(['Client Unique ID', 'EE Provider ID','Entry Exit Group Id','Entry Exit Household Id','Unnamed: 6','Housing Move-in Date(5584)'],1)
entry_exit.head()

In [ ]:
entry_exit['Destination']=entry_exit['Destination'].replace("Client doesn't know (HUD)", np.NaN).replace("Client refused (HUD)", np.NaN).replace("Data not collected (HUD)", np.NaN)
entry_exit.groupby("Destination").size()

In [ ]:
entry_exit.groupby("Reason for Leaving").size()

In [ ]:
entry_exit.groupby("Entry Exit Type").size()

In [ ]:
entry_exit = entry_exit.drop(['Entry Exit Type','Entry Exit Date Added','Entry Exit Date Updated'],1)
entry_exit.head()

In [ ]:
entry_exit[['Entry Date', 'Exit Date']] = entry_exit[['Entry Date', 'Exit Date']].apply(pd.to_datetime)
entry_exit['stay_days']=entry_exit['Exit Date'] - entry_exit['Entry Date']
entry_exit.head()

### Import data: ee_udes

In [ ]:
ee_udes = pd.read_csv("../data/EE_UDES_191102.tsv", delimiter='\t', encoding='utf-8')
ee_udes.head()

In [ ]:
ee_udes=ee_udes[ee_udes["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']

In [ ]:
ee_udes = ee_udes.drop(['Client Unique ID', 'EE Provider ID'],1)
ee_udes.head()

In [ ]:
#How many rows in dataframe?
ee_udes.shape[0]

In [ ]:
ee_udes.groupby("Did you stay less than 7 nights?(5164)").size()

In [ ]:
ee_udes.groupby("Did you stay less than 90 days?(5163)").size()

In [ ]:
ee_udes.groupby("On the night before did you stay on the streets, ES or SH?(5165)").size()

In [ ]:
ee_udes.groupby("If yes for Domestic violence victim/survivor, when experience occurred(1917)").size()

Drop four columns above since there are too many missing values.

In [ ]:
ee_udes = ee_udes.drop(['Did you stay less than 7 nights?(5164)', 'Did you stay less than 90 days?(5163)','On the night before did you stay on the streets, ES or SH?(5165)','If yes for Domestic violence victim/survivor, when experience occurred(1917)'],1)
ee_udes.head()

In [ ]:
ee_reviews = pd.read_csv("../data/EE_REVIEWS_191102.tsv", delimiter='\t', encoding='utf-8')
ee_reviews.head()

In [ ]:
client["EE UID"].nunique()

In [ ]:
entry_exit["EE UID"].nunique()

In [ ]:
ee_udes["EE UID"].nunique()

### Merge data
The data frames of "client" "entry_exit" and "ee_udes" have the same number of "EE UID" values. Join them on this key.

In [ ]:
merged_data=pd.merge(pd.merge(client, entry_exit, how='left', on='EE UID'),ee_udes,how='left', on='EE UID')
merged_data = merged_data.drop(['Client ID_y', 'Client ID'],1)
df.rename(columns={"Client ID_x": "Client ID"})
merged_data

Keep only clients' first entry and merge data frames on "Client ID".

In [ ]:
client_first = client.drop_duplicates(subset='Client ID', keep='first')
client_first = client_first.drop(['EE UID'],1)
entry_exit_first=entry_exit.drop_duplicates(subset='Client ID', keep='first')
entry_exit_first = entry_exit_first.drop(['EE UID'],1)
ee_udes_first=ee_udes.drop_duplicates(subset='Client ID', keep='first')
ee_udes_first = ee_udes_first.drop(['EE UID'],1)
merged_data_client=pd.merge(pd.merge(client_first, entry_exit_first, how='left', on='Client ID'),ee_udes_first,how='left', on='Client ID')
merged_data_client

In [ ]:
merged_data.to_csv("../data/merged_data.tsv", sep='\t')

In [ ]:
merged_data_client.to_csv("../data/merged_data_client.tsv", sep='\t')

In [ ]:
for column in merged_data.columns: 
    print(column)

In [ ]:
all_sort = merged_data.sort_values(by=['Entry Date'], ascending=True)
all_sort

### Import data: health_ins_entry and health_ins_exit

In [ ]:
import pandas as pd
health_ins_entry = pd.read_csv("../data/HEALTH_INS_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')
health_ins_entry.head()

In [ ]:
health_ins_exit = pd.read_csv("../data/HEALTH_INS_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
health_ins_exit.head()

In [ ]:
health_ins_entry=health_ins_entry[health_ins_entry["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']
health_ins_exit=health_ins_exit[health_ins_exit["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']
health_ins_entry = health_ins_entry.drop(['Client Unique ID', 'EE Provider ID','Recordset ID (4307-recordset_id)','Date Added (4307-date_added)'],1)
health_ins_exit = health_ins_exit.drop(['Client Unique ID', 'EE Provider ID','Recordset ID (4307-recordset_id)','Date Added (4307-date_added)'],1)


Keep only clients' first entry and merge data frames on "Client ID".

In [ ]:
health_ins_entry_first=health_ins_entry.drop_duplicates(subset='Client ID', keep='first')
health_ins_entry_first = health_ins_entry_first.drop(['EE UID'],1)
health_ins_exit_first=health_ins_exit.drop_duplicates(subset='Client ID', keep='first')
health_ins_exit_first = health_ins_exit_first.drop(['EE UID'],1)
merged_data_health_ins=pd.merge(health_ins_entry_first, health_ins_exit_first, how='left', on='Client ID')
merged_data_health_ins.to_csv("../data/merged_data_health_ins.tsv", sep='\t')


In [ ]:
merged_data_health_ins.head()

### Import data: income_entry and income_exit

In [1]:
import pandas as pd
income_entry = pd.read_csv("../data/INCOME_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')
income_entry.head()

,EE Provider ID,EE UID,Client Unique ID,Client ID,Receiving Income (Entry),Income Source (Entry),Monthly Amount (Entry),Income Start Date (Entry),Income End Date (Entry),Recordset ID (140-recordset_id),Provider (140-provider),Date Added (140-date_added)
0,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Alimony or Other Spousal Support (HUD),NaN,4/20/2015,NaN,3263585,Urban Ministries of Durham - Durham County - S...,12/31/2015
1,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Child Support (HUD),NaN,4/20/2015,NaN,3263586,Urban Ministries of Durham - Durham County - S...,12/31/2015
2,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Earned Income (HUD),NaN,4/20/2015,NaN,3263590,Urban Ministries of Durham - Durham County - S...,12/31/2015
3,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,General Assistance (HUD),NaN,4/20/2015,NaN,3263587,Urban Ministries of Durham - Durham County - S...,12/31/2015
4,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Other (HUD),NaN,4/20/2015,NaN,3263599,Urban Ministries of Durham - Durham County - S...,12/31/2015


In [2]:
income_exit = pd.read_csv("../data/INCOME_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
income_exit.head()

,EE Provider ID,EE UID,Client Unique ID,Client ID,ReceivingIncome (Exit),Source of Income (Exit),Monthly Amount (Exit),Income Start Date (Exit),Income End Date (Exit),Recordset ID (140-recordset_id),Provider (140-provider),Date Added (140-date_added)
0,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Alimony or Other Spousal Support (HUD),NaN,4/20/2015,NaN,3263585,Urban Ministries of Durham - Durham County - S...,12/31/2015
1,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Child Support (HUD),NaN,4/20/2015,NaN,3263586,Urban Ministries of Durham - Durham County - S...,12/31/2015
2,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Earned Income (HUD),NaN,4/20/2015,NaN,3263590,Urban Ministries of Durham - Durham County - S...,12/31/2015
3,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,General Assistance (HUD),NaN,4/20/2015,NaN,3263587,Urban Ministries of Durham - Durham County - S...,12/31/2015
4,Urban Ministries of Durham - Durham County - S...,687901,pbkf09291954p610b236,397941,No,Other (HUD),NaN,4/20/2015,NaN,3263599,Urban Ministries of Durham - Durham County - S...,12/31/2015


In [4]:
income_entry=income_entry[income_entry["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']
income_exit=income_exit[income_exit["EE Provider ID"]=='Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)']
income_entry = income_entry.drop(['Client Unique ID', 'EE Provider ID','Recordset ID (140-recordset_id)','Date Added (140-date_added)'],1)
income_exit = income_exit.drop(['Client Unique ID', 'EE Provider ID','Recordset ID (140-recordset_id)','Date Added (140-date_added)'],1)

income_entry_first=income_entry.drop_duplicates(subset='Client ID', keep='first')
income_entry_first = income_entry_first.drop(['EE UID'],1)
income_exit_first=income_exit.drop_duplicates(subset='Client ID', keep='first')
income_exit_first = income_exit_first.drop(['EE UID'],1)
merged_data_income=pd.merge(income_entry_first, income_exit_first, how='left', on='Client ID')
merged_data_income.to_csv("../data/merged_data_income.tsv", sep='\t')
merged_data_income.head()

,Client ID,Receiving Income (Entry),Income Source (Entry),Monthly Amount (Entry),Income Start Date (Entry),Income End Date (Entry),Provider (140-provider)_x,ReceivingIncome (Exit),Source of Income (Exit),Monthly Amount (Exit),Income Start Date (Exit),Income End Date (Exit),Provider (140-provider)_y
0,397941,No,Alimony or Other Spousal Support (HUD),NaN,4/20/2015,NaN,Urban Ministries of Durham - Durham County - S...,No,Alimony or Other Spousal Support (HUD),NaN,4/20/2015,NaN,Urban Ministries of Durham - Durham County - S...
1,130335,No,Alimony or Other Spousal Support (HUD),NaN,6/13/2015,NaN,Urban Ministries of Durham - Durham County - S...,No,Alimony or Other Spousal Support (HUD),NaN,6/13/2015,NaN,Urban Ministries of Durham - Durham County - S...
2,188933,No,Alimony or Other Spousal Support (HUD),NaN,3/28/2014,NaN,Urban Ministries of Durham - Durham County - S...,No,Alimony or Other Spousal Support (HUD),NaN,3/28/2014,NaN,Urban Ministries of Durham - Durham County - S...
3,168290,No,Alimony or Other Spousal Support (HUD),NaN,10/6/2014,NaN,Urban Ministries of Durham - Durham County(1562),No,Alimony or Other Spousal Support (HUD),NaN,10/6/2014,NaN,Urban Ministries of Durham - Durham County(1562)
4,123122,No,Alimony or Other Spousal Support (HUD),NaN,8/1/2013,NaN,Urban Ministries of Durham - Durham County(1562),No,Alimony or Other Spousal Support (HUD),NaN,8/1/2013,NaN,Urban Ministries of Durham - Durham County(1562)
